In [1]:
from sympy import *
import random
init_printing()
from sympy.physics.quantum.dagger import Dagger

In [2]:
def randMatrixC(r,c,min=0,max=1,seed=[]):
    """Create random matrix r x c with complex entries"""
    if seed == []:
        prng = random.Random()  # use system time
    else:
        prng = random.Random(seed)
    return Matrix(r,c,lambda i,j: prng.randint(min,max)+I*prng.randint(min,max))

In [3]:
def MTS(A):
    """Calcula una matriz triangular superior"""
    A1=A
    n=A.shape[0]
    V=eye(n)
    for i in range(0,n):
        C=eye(n)
        C[i,i]=1/sqrt(A1[i,i])
        for j in range(i+1,n):
            C[i,j] = -(1/A1[i,i])*A1[i,j]
        V=V*C
        V.simplify()
        A1=Dagger(C)*A1*C
        A1.simplify()
    return V

In [4]:
A=zeros(2,2)
A[0,0]=2
A[0,1]=1
A[1,0]=1
A[1,1]=2
A,MTS(A)

In [5]:
A=randMatrixC(6,6)
B=Dagger(A)*A
B.simplify()
B

In [6]:
C=MTS(B)
C

In [7]:
J=Dagger(C)*B*C
J.simplify()
J